In [ ]:
import geopandas as gpd
from ribasim_nl import CloudStorage
from ribasim_nl.geodataframe import join_by_poly_overlay, split_basins

cloud = CloudStorage()

# Opknippen KRW polygonen met lijnen
- Opsplitsen van `krw_oppervlaktewaterlichamen_nederland_vlakken.gpkg` met `krw_split_lijnen.gpkg`
- Toevoegen van `waterlichaam` namen uit `oppervlaktewaterlichamen_rijk.gpkg`

In [ ]:
exclude_owmident = [
    "NL95_1A",
    "NL95_2A",
    "NL95_3A",
    "NL95_4A",
    "NL81_1",
    "NL81_10",
    "NL81_3",
    "NL81_2",
    "NL89_zwin",
]
krw_poly_gdf = gpd.read_file(
    cloud.joinpath(
        "Basisgegevens", "KRW", "krw_oppervlaktewaterlichamen_nederland_vlakken.gpkg"
    )
)

krw_poly_gdf = krw_poly_gdf.loc[~krw_poly_gdf.owmident.isin(exclude_owmident)]
krw_poly_gdf = krw_poly_gdf.explode(index_parts=False)
krw_poly_gdf["shapeArea"] = krw_poly_gdf.area
krw_poly_gdf.sort_values("shapeArea", ascending=False)
krw_poly_gdf.drop_duplicates("owmident", keep="first", inplace=True)


krw_split_lines_gdf = gpd.read_file(
    cloud.joinpath("Rijkswaterstaat", "verwerkt", "model_user_data.gpkg"),
    layer="krw_split_lijnen",
)


rws_opp_poly_gdf = gpd.read_file(
    cloud.joinpath(
        "Rijkswaterstaat", "aangeleverd", "oppervlaktewaterlichamen_rijk.gpkg"
    )
)


rws_krw_poly_gdf = split_basins(krw_poly_gdf, krw_split_lines_gdf)


rws_krw_poly_gdf = join_by_poly_overlay(
    rws_krw_poly_gdf,
    rws_opp_poly_gdf[["waterlichaam", "geometry"]],
    select_by="poly_area",
)


rws_krw_poly = cloud.joinpath("Rijkswaterstaat", "verwerkt", "krw_basins_vlakken.gpkg")


rws_krw_poly_gdf.to_file(rws_krw_poly)